In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install required packages
!pip install gradio opencv-python-headless pillow

# Import all libraries
import torch
import torch.nn as nn
from torchvision.models import resnext50_32x4d, ResNeXt50_32X4D_Weights
from torchvision import transforms
import cv2
import numpy as np
import gradio as gr
from PIL import Image
import tempfile
import os

# Define your model architecture
HIDDEN_SIZE = 256

class DeepfakeLSTMModel(nn.Module):
    def __init__(self, hidden_size=HIDDEN_SIZE, num_layers=1, num_classes=2):
        super().__init__()
        self.backbone = resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        self.lstm = nn.LSTM(
            input_size=in_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        batch_size, seq_len, c, h, w = x.size()
        x = x.view(batch_size * seq_len, c, h, w)
        features = self.backbone(x)
        features = features.view(batch_size, seq_len, -1)
        lstm_out, _ = self.lstm(features)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Load model
print("🔧 Loading model...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = DeepfakeLSTMModel()
model.load_state_dict(torch.load('/content/drive/MyDrive/deepfake_lstm_trained.pth', map_location=device))
model.to(device)
model.eval()
print("✅ Model loaded successfully!")

# Image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Extract frames function with preview images
def extract_frames_with_preview(video_path, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        cap.release()
        return None, []

    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    frames = []
    preview_images = []

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Save for preview (original size)
            preview_images.append(Image.fromarray(frame_rgb))

            # Process for model
            frame_tensor = transform(frame_rgb)
            frames.append(frame_tensor)

    cap.release()

    if len(frames) == 0:
        return None, []

    frames_tensor = torch.stack(frames).unsqueeze(0)
    return frames_tensor, preview_images

# Create a grid of frames for display
def create_frame_grid(images, max_display=6):
    """Create a grid of extracted frames"""
    if not images:
        return None

    # Take only first max_display frames
    display_images = images[:max_display]

    # Calculate grid dimensions
    n = len(display_images)
    cols = 3
    rows = (n + cols - 1) // cols

    # Get individual image size
    img_width, img_height = display_images[0].size

    # Create grid
    grid_width = img_width * cols + 20 * (cols - 1)
    grid_height = img_height * rows + 20 * (rows - 1)
    grid = Image.new('RGB', (grid_width, grid_height), color=(30, 30, 30))

    # Paste images
    for idx, img in enumerate(display_images):
        row = idx // cols
        col = idx % cols
        x = col * (img_width + 20)
        y = row * (img_height + 20)
        grid.paste(img, (x, y))

    return grid

# Prediction function
def predict_deepfake(video_file, sequence_length):
    if video_file is None:
        return "❌ Please upload a video file", None, None, None

    try:
        print(f"📹 Processing video: {video_file}")
        print(f"🎬 Sequence length: {sequence_length}")

        # Extract frames with preview
        print("⏳ Extracting frames...")
        frames, preview_images = extract_frames_with_preview(video_file, num_frames=sequence_length)

        if frames is None:
            return "❌ Failed to extract frames from video", None, None, None

        # Create frame grid for display
        frame_grid = create_frame_grid(preview_images, max_display=6)

        # Make prediction
        print("🤖 Running model prediction...")
        with torch.no_grad():
            frames = frames.to(device)
            outputs = model(frames)
            probabilities = torch.softmax(outputs, dim=1)
            confidence, predicted = torch.max(probabilities, 1)

            prediction = 'FAKE' if predicted.item() == 0 else 'REAL'
            confidence_score = confidence.item()

        print(f"✅ Prediction: {prediction} ({confidence_score:.2%})")

        # Format results
        if prediction == 'REAL':
            result_text = f"✅ AUTHENTIC VIDEO\n\nConfidence: {confidence_score*100:.1f}%"
        else:
            result_text = f"❌ DEEPFAKE DETECTED\n\nConfidence: {confidence_score*100:.1f}%"

        # Create confidence breakdown
        fake_conf = probabilities[0][0].item()
        real_conf = probabilities[0][1].item()

        confidence_dict = {
            "Fake": fake_conf,
            "Real": real_conf
        }

        details = f"""
## 📊 Analysis Details

- **Prediction:** {prediction}
- **Confidence:** {confidence_score*100:.1f}%
- **Frames Analyzed:** {sequence_length}
- **Model:** LSTM + ResNeXt50
- **Device:** {device}

### 🔍 Probability Breakdown:
- **Fake:** {fake_conf*100:.1f}%
- **Real:** {real_conf*100:.1f}%

### 🎬 Frame Extraction:
The model analyzed {len(preview_images)} frames evenly distributed throughout the video.
Above you can see a preview of the first 6 frames that were processed.
        """

        return result_text, confidence_dict, details, frame_grid

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return f"❌ Error: {str(e)}", None, None, None

# Create Gradio Interface
print("🎨 Creating Gradio interface...")

with gr.Blocks(theme=gr.themes.Soft(primary_hue="purple")) as demo:
    gr.Markdown("""
    # 🎬 Deepfake Detector
    ### Advanced LSTM-based Video Analysis
    Upload a video file to analyze it for deepfake detection using ResNeXt50 + LSTM model.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            # Simple file upload
            video_input = gr.File(
                label="📹 Upload Video File",
                file_types=["video"],
                type="filepath"
            )

            sequence_slider = gr.Slider(
                minimum=5,
                maximum=30,
                value=10,
                step=1,
                label="🎬 Sequence Length (Number of frames to analyze)",
                info="Higher values = more accurate but slower"
            )

            analyze_btn = gr.Button("🚀 Analyze Video", variant="primary", size="lg")

        with gr.Column(scale=1):
            result_text = gr.Textbox(label="🎯 Result", lines=3, show_label=True)
            confidence_plot = gr.Label(label="📊 Confidence Scores", num_top_classes=2)

    # Frame preview section
    with gr.Row():
        frame_gallery = gr.Image(label="🎞️ Extracted Frames Preview (First 6)", show_label=True)

    # Details section
    with gr.Row():
        details_text = gr.Markdown(label="📋 Analysis Details")

    gr.Markdown(f"""
    ---
    ### ℹ️ Model Information
    - **Architecture:** LSTM + CNN
    - **Hidden Size:** 256
    - **Classes:** Real / Fake
    - **Running on:** {device}

    ### 📝 How to use:
    1. Click "Upload Video File" and select your video
    2. Adjust sequence length (default: 10 frames)
    3. Click "🚀 Analyze Video"
    4. Wait 30-60 seconds for results
    5. View extracted frames to see what the model analyzed

    ### ⚡ Tips:
    - Supported formats: MP4, AVI, MOV, MKV
    - Longer videos may take more time
    - Higher sequence length = more accurate but slower
    - Frames are extracted evenly throughout the video
    """)

    # Connect the analyze button
    analyze_btn.click(
        fn=predict_deepfake,
        inputs=[video_input, sequence_slider],
        outputs=[result_text, confidence_plot, details_text, frame_gallery]
    )

# Launch the interface
print("\n" + "="*60)
print("🚀 Launching Gradio Interface...")
print("="*60)

demo.launch(share=True, debug=True)

🔧 Loading model...
Using device: cuda
✅ Model loaded successfully!
🎨 Creating Gradio interface...

🚀 Launching Gradio Interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://917b986bb86b4d5d16.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

📹 Processing video: /tmp/gradio/4532c47fc1809a5f7f1b835b880845818894c13d912498135e4ca1916d3c384a/00066.mp4
🎬 Sequence length: 10
⏳ Extracting frames...
🤖 Running model prediction...
✅ Prediction: REAL (97.07%)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

📹 Processing video: /tmp/gradio/2b1a33187be5cb30ab0491868ee2d5f32253903592d4020aa48c7f3cbc9c2055/id0_id2_0002.mp4
🎬 Sequence length: 10
⏳ Extracting frames...
🤖 Running model prediction...
✅ Prediction: FAKE (71.74%)
